In [1]:
!pip install pymc==2.3.6
import pymc as pm
import numpy as np
import spacy

import matplotlib.colors as mcolors

!pip install plotly
import plotly.graph_objects as go
from plotly.colors import n_colors

from PIL import ImageColor

from scipy.stats import wasserstein_distance
from scipy.stats import entropy

from os import listdir
from os.path import join, isfile

COLORS = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
RGBCOLORS = [str(ImageColor.getrgb(c[1])) for c in mcolors.TABLEAU_COLORS.items()]

## Text preprocessing

First, **upload corpuses.zip** to Google Colab in /content directory.

In [2]:
#@title Unzip dataset archive
!unzip /content/corpuses.zip

Archive:  /content/corpuses.zip
replace corpuses/art_and_science/document_01? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: corpuses/art_and_science/document_01  
  inflating: corpuses/art_and_science/document_02  
  inflating: corpuses/art_and_science/document_03  
  inflating: corpuses/art_and_science/document_04  
  inflating: corpuses/art_and_science/document_05  
  inflating: corpuses/boots_cats_bees/document1  
  inflating: corpuses/boots_cats_bees/document2  
  inflating: corpuses/boots_cats_bees/document3  
  inflating: corpuses/boots_cats_bees/document4  
  inflating: corpuses/boots_cats_bees/document5  
  inflating: corpuses/boots_cats_bees/document6  
  inflating: corpuses/boots_cats_bees/document7  
  inflating: corpuses/boots_cats_bees/document8  
  inflating: corpuses/boots_cats_bees/document9  
  inflating: corpuses/nuts_and_animals/document_01  
  inflating: corpuses/nuts_and_animals/document_02  
  inflating: corpuses/nuts_and_animals/document_03  
  inflating: 

In [3]:
#@title Define preprocessing functions
class TextProcessor:
    nlp = spacy.load('en_core_web_sm')

    def __init__(self):
        self.docs = []
        self.processed_docs = []

        self.vocabulary = []
        self.word2index = {}

    def load_corpus(self, corpus_path):
        for file in sorted(listdir(corpus_path)):
            file_path = join(corpus_path, file)
            if isfile(file_path):
                with open(file_path, 'r') as doc:
                    self.docs.append(TextProcessor.nlp(doc.read()))

    def process_documents(self):
        _vocabulary = set()

        for doc in self.docs:
            processed_doc = []

            # Filter out punctuation and stopwords
            tokens = list(filter(lambda token: not token.is_punct and not token.is_stop, doc))

            # Lemmatization
            tokens = list(map(lambda token: token.lemma_.lower(), tokens))

            for token in tokens:
                processed_doc.append(token)
                _vocabulary.add(token)

            self.processed_docs.append(processed_doc)

        for it, token in enumerate(sorted(_vocabulary)):
            self.vocabulary.append(token)
            self.word2index[token] = it


text_processor = TextProcessor()
text_processor.load_corpus(corpus_path='/content/corpuses/nuts_and_animals')
text_processor.process_documents()

print('Vocabulary of "nuts_and_animals":\n\n', text_processor.vocabulary, '\n\n')
print(f'Tokenized docs:\n')
text_processor.processed_docs

Vocabulary of "nuts_and_animals":

 ['almond', 'breakfast', 'butter', 'cat', 'dog', 'eat', 'enemy', 'feed', 'get', 'like', 'little', 'mortal', 'neighbor', 'peanut', 'sandwich', 'walnut', 'yesterday'] 


Tokenized docs:



[['peanut', 'butter', 'sandwich', 'breakfast'],
 ['like', 'eat', 'almond', 'peanut', 'walnut'],
 ['neighbor', 'get', 'little', 'dog', 'yesterday'],
 ['cat', 'dog', 'mortal', 'enemy'],
 ['feed', 'peanut', 'dog']]

## Latent Dirichlet Allocation

In [6]:
class LDA:
    def __init__(self, num_topics, corpus_path, alpha=None, beta=None):
        self.num_topics = num_topics

        # Load and preprocess the documents
        self.text_processor = TextProcessor()
        self.text_processor.load_corpus(corpus_path=corpus_path)
        self.text_processor.process_documents()

        self.size_vocabulary = len(self.text_processor.vocabulary)
        self.num_documents = len(self.text_processor.processed_docs)

        # Initialize alpha and beta to 1 if not given
        if beta is None:
            self.beta = np.ones(self.size_vocabulary) / self.num_topics
        else:
            self.beta = beta

        if alpha is None:
            self.alpha = np.ones(self.num_topics) / self.num_topics
        else:
            self.alpha = alpha

        # Define random variables
        self.phi = pm.Container([pm.CompletedDirichlet(name=f'cphi_{k}', D=pm.Dirichlet(f'phi_{k}', theta=self.beta))
                                 for k in range(self.num_topics)])
        self.theta = pm.Container([pm.CompletedDirichlet(name=f'ctheta_{m}', D=pm.Dirichlet(f'theta_{m}', theta=self.alpha))
                                   for m in range(self.num_documents)])

        self.z = pm.Container([[pm.Categorical(name=f'z_{m}_{n}', p=self.theta[m])
                                for n in range(len(self.text_processor.processed_docs[m]))] for m in range(self.num_documents)])
        self.w = pm.Container([[pm.Categorical(name=f'w_{m}_{n}',
                                  p=pm.Lambda(f'pw_{m}_{n}', lambda z_=self.z[m][n], phi_=self.phi: phi_[z_]),
                                  value=self.text_processor.word2index[self.text_processor.processed_docs[m][n]],
                                  observed=True)
                   for n in range(len(self.text_processor.processed_docs[m]))] for m in range(self.num_documents)])

        # Define model and MCMC
        self.model = pm.Model([self.phi, self.theta, self.z, self.w])
        self.mcmc = pm.MCMC(self.model)

    def fit(self, iter=20000, burn=5000):
        self.mcmc.sample(iter, burn, 1)

        # Collect samples from MCMC
        self.phi_trace = np.array([self.mcmc.trace(f'cphi_{m}')[:].squeeze(axis=1) for m in range(self.num_topics)])
        self.theta_trace = np.array([self.mcmc.trace(f'ctheta_{m}')[:].squeeze(axis=1) for m in range(self.num_documents)])

        # Average results
        self.phi_trace = self.phi_trace.mean(axis=1)
        self.theta_trace = self.theta_trace.mean(axis=1)

        return self.phi_trace, self.theta_trace
    
    def draw_table(self, table_columns, table_rows, table_colors):
        # Create figure
        fig = go.Figure(data=go.Table(
            columnwidth=[300, 100],
            header=dict(values=table_columns,
                        line_color='white',
                        fill_color='white',
                        align='center', font=dict(color='black', size=12),
                        height=30),
            cells=dict(values=table_rows,
                       align='center',
                       font=dict(color=['black', 'white'], size=11),
                       fill_color=table_colors)))
        
        # Adjust layout to match table real size
        fig.update_layout(margin=dict(l=0, r=0, b=0, t=0, pad=0, autoexpand=False))
        fig.update_layout(height=50 + 20 * len(table_rows[0]))

        # Something beautiful will appear
        fig.show()

    def show_documents(self):
        table_columns = ['Document'] + [f'Topic #{i}' for i in range(1, self.num_topics + 1)]
        table_rows = []
        table_colors = []

        for doc_id in range(self.num_documents):
            # Take document intro
            doc_head = self.text_processor.docs[doc_id].text[:65]
            if len(self.text_processor.docs[doc_id].text) > 65:
                doc_head += '[...]'
            
            # Select topic assignment
            values = []
            colors = ['rgba(230, 216, 110, 0.1)']
            for topic_id in range(self.num_topics):
                values.append(f'{self.theta_trace[doc_id][topic_id]:.2f}')
                alpha = f'{self.theta_trace[doc_id][topic_id]:.2f}'
                colors.append(f'rgba{RGBCOLORS[topic_id][:-1]}, {alpha})')
            
            # Construct table row
            table_rows.append([doc_head] + values)
            table_colors.append(colors)
        
        # Transpose values
        table_rows = np.array(table_rows).T.tolist()
        table_colors = np.array(table_colors).T.tolist()

        self.draw_table(table_columns, table_rows, table_colors)

    def show_top_words():
        pass

    def show_words(self):
        table_columns = ['Word'] + [f'Topic #{i}' for i in range(1, self.num_topics + 1)]
        table_rows = []
        table_colors = []

        for word_id in range(self.size_vocabulary):
            # Select word
            word = self.text_processor.vocabulary[word_id]

            # Select topic assignment
            values = []
            colors = ['rgba(230, 216, 110, 0.1)']
            for topic_id in range(self.num_topics):
                values.append(f'{self.phi_trace[topic_id][word_id]:.2f}')
                alpha = f'{min(1.5 * self.phi_trace[topic_id][word_id], 1):.2f}'
                colors.append(f'rgba{RGBCOLORS[topic_id][:-1]}, {alpha})')
            
            # Construct table row
            table_rows.append([word] + values)
            table_colors.append(colors)

        # Transpose values
        table_rows = np.array(table_rows).T.tolist()
        table_colors = np.array(table_colors).T.tolist()

        self.draw_table(table_columns, table_rows, table_colors)

## Results

### Sanity check #1

In [5]:
lda = LDA(num_topics=2, corpus_path='/content/corpuses/sanity_check/')
lda.fit(10000, 5000)
lda.show_documents()
lda.show_words()

 [-----------------100%-----------------] 10000 of 10000 complete in 26.5 sec

### Sanity Check #2 (Multiple Topics)

In [8]:
lda = LDA(num_topics=4, corpus_path='/content/corpuses/sanity_check_multiple_topics/')
lda.fit(5000, 1000)
lda.show_documents()
lda.show_words()

 [-----------------100%-----------------] 5000 of 5000 complete in 34.6 sec

### Nuts and Animals

In [9]:
lda = LDA(num_topics=2, corpus_path='/content/corpuses/nuts_and_animals/')
lda.fit(10000, 2000)
lda.show_documents()
lda.show_words()

 [-----------------100%-----------------] 10000 of 10000 complete in 28.0 sec

### Art and Science

In [11]:
lda = LDA(num_topics=2, corpus_path='/content/corpuses/art_and_science//')
lda.fit(10000, 2000)
lda.show_documents()
lda.show_words()

 [-----------------100%-----------------] 10000 of 10000 complete in 289.8 sec

### Boots'n'Cats + Bees

In [15]:
lda = LDA(num_topics=3, corpus_path='/content/corpuses/boots_cats_bees/')
lda.fit(5000, 2000)
lda.show_documents()
lda.show_words()

 [-----------------100%-----------------] 5000 of 5000 complete in 72.3 sec

## Topic based similarity metric

### Earth Mover's Distance (1st Wasserstein distance)

Earth Mover's Distance is a metric that describes the cost required to transform one probability distribution into another.

We use $\theta_{m}$, $1 \leq m \leq M$ that describe each document's assignment over the $K$ topics as our metric space.

We compute $dist(document_i, document_j)$ as $EMD(\theta_i, \theta_j)$ for $1 \leq i \leq M$, $1 \leq j \leq M$ and then plot the distances matrix.



In [16]:
def compute_EDA(lda):
    distances = []
    for i in range(lda.num_documents):
        distances.append([wasserstein_distance(lda.theta_trace[i], lda.theta_trace[j]) for j in range(lda.num_documents)])
    
    return np.array(distances)

distances = compute_EDA(lda)

In [17]:
def plot_distances(lda, distances, color_index=6):
    table_columns = ['',]
    table_rows = []
    table_colors = []
    
    for doc_id1 in range(lda.num_documents):
        # Take 1st document intro
        doc_head1 = lda.text_processor.docs[doc_id1].text[:10]
        if len(lda.text_processor.docs[doc_id1].text) > 10:
            doc_head1 += '[...]'
        
        table_columns.append(doc_head1)

        # Select topic assignment
        values = [doc_head1]
        colors = ['rgba(230, 216, 110, 0.1)']
        for doc_id2 in range(lda.num_documents):
            values.append(f'{distances[doc_id1][doc_id2]:.2f}')
            alpha = f'{min(3.5 * distances[doc_id1][doc_id2], 1):.2f}'
            colors.append(f'rgba{RGBCOLORS[color_index + (doc_id2 // 3)][:-1]}, {alpha})')
        
        # Construct table row
        table_rows.append(values)
        table_colors.append(colors)
    
    # Transpose values
    table_rows = np.array(table_rows).T.tolist()
    table_colors = np.array(table_colors).T.tolist()

    # Create figure
    fig = go.Figure(data=go.Table(
        columnwidth=[100, 100],
        header=dict(values=table_columns,
                    line_color='rgba(230, 216, 110, 0.1)',
                    fill_color='rgba(230, 216, 110, 0.1)',
                    align='center', font=dict(color='black', size=11)),
        cells=dict(values=table_rows,
                    align='center',
                    font=dict(color=['black', 'white'], size=11),
                    fill_color=table_colors)))
    
    # Adjust layout to match table real size
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0, pad=0, autoexpand=False))
    fig.update_layout(height=50 + 20 * len(table_rows[0]))

    # Something beautiful will appear
    fig.show()


plot_distances(lda, distances)

### Kullback–Leibler divergence (relative entropy)

Similarly, we will compute the Kullback–Leibler divergence, a measure that characterizes how much a distribution differs from another.

We have: $dist(document_i, document_j)$ as $D_{KL}(\theta_i, \theta_j)$ for $1 \leq i \leq M$, $1 \leq j \leq M$.

In [18]:
def compute_DKL(lda):
    distances = []
    for i in range(lda.num_documents):
        distances.append([entropy(lda.theta_trace[i], lda.theta_trace[j]) for j in range(lda.num_documents)])
    
    return np.array(distances)

distances = compute_EDA(lda)

In [19]:
plot_distances(lda, distances)

## Topic assignment for new document

We want to infer the topic assignment for a new document $\theta_{new}$ based on the known probability distributions from a previous training.

We apply the following steps:

<ol>
    <li>Split the new document into tokens</li>
    <li>For each token $i$, determine its contribution to the topic assignment:
        <ul>
            <li>Use the per-topic word distribution $\varphi_k$ to determine how much token $i$ contributes to each topic $k$. </li>
            <li>Add this value to the total assignment</li>
        </ul>
    <li>Divide by the number of tokens to normalize the assignment.</li>
</ol>

Therefore, we have:
$\theta_{new} = \frac{1}{N} \sum_{i=1}^{N} \varphi_{index(token_i)}$.

In [20]:
def assign_topic(tokenized_doc, lda):
    topic_assignment = []
    for token in tokenized_doc:
        index = lda.text_processor.word2index[token]
        topic_assignment.append(lda.phi_trace.T[index])
    
    # Add all values
    topic_assignment = np.array(topic_assignment).sum(axis=0)
    
    # Normalize
    topic_assignment /= topic_assignment.sum()

    return topic_assignment

### Pretraining

In [22]:
lda = LDA(num_topics=2, corpus_path='/content/corpuses/sanity_check/')
lda.fit(10000, 5000)
lda.show_documents()
lda.show_words()

 [-----------------100%-----------------] 10000 of 10000 complete in 28.9 sec

### Inference

In [23]:
# Declare new documents for topic assignment
docs = ['aaa bbb aaa aaa aaa bbb',
        'vvv uuu vvv uuu uuu vvv',
        'aaa vvv uuu bbb']

# Inference
theta_new = []
for doc in docs:
    theta_new.append(assign_topic(doc.split(' '), lda))

# Overwrite in existing lda object, very hacky :(
lda.num_documents = len(docs)
lda.theta_trace = np.array(theta_new)
class Doc(object):
    pass
extended_docs = []
for i in range(len(docs)):
    doc = Doc()
    doc.text = docs[i]
    extended_docs.append(doc)
lda.text_processor.docs = extended_docs

# Show results for new documents
lda.show_documents()